In [85]:
# Python 3.9 
import geopandas as gpd
import simplekml
from shapely.ops import transform

In [89]:
def round_coords_and_flatten(geom, ndigits=2):
    
   def _round_coords(x, y, z=None):
      x = round(x, ndigits)
      y = round(y, ndigits)

      return (x,y)
   
   return transform(_round_coords, geom)

In [90]:
routes = gpd.read_file("./Daten/veloland.gdb",layer="Route")

# Reproject geodataframe from CH1903+ LV95 (EPSG:2056) to WGS1984 (EPSG:4326)
routes = routes.to_crs("epsg:4326")

routes["geometry"] = routes.geometry.apply(round_coords_and_flatten, ndigits=5)

In [131]:
# Define styles
shared_style_international = simplekml.Style()
shared_style_international.linestyle.width = 8
shared_style_international.linestyle.color = "cc181D9D"

shared_style_national = simplekml.Style()
shared_style_national.linestyle.width = 6
shared_style_national.linestyle.color = "cc181DCD"

shared_style_regional = simplekml.Style()
shared_style_regional.linestyle.width = 5
shared_style_regional.linestyle.color = "cc055CFD"

shared_style_lokal = simplekml.Style()
shared_style_lokal.linestyle.width = 4
shared_style_lokal.linestyle.color = "cc078FE6"

shared_style_other = simplekml.Style()
shared_style_other.linestyle.width = 4
shared_style_other.linestyle.color = "cce75b15"

styles = {
    "International": shared_style_international,
    "National": shared_style_national,
    "Regional": shared_style_regional,
    "Lokal": shared_style_lokal
}

In [133]:
for ttype in ["International", "National", "Regional", "Lokal"]:    
    kml = simplekml.Kml()
    kml.document.name = "Velorouten %s – by ralphstraumann.ch" % ttype.lower()

    routes_ = [route for route in routes.iterfeatures()]
    for route in routes_:#[0:1]:
        geom = route.get("geometry").get("coordinates")[0]

        props = route.get("properties")
        tour_nr = props.get("TourNrR")
        tour_name = props.get("TourNameR")
        tour_desc = props.get("BeschreibR")    
        tour_start = props.get("AOrt")
        tour_end = props.get("ZOrt")
        tour_dir = props.get("Abwicklung")

        tour_dist_km = props.get("LaengeR") / 1000
        tour_cond_req = props.get("KonditionR")
        tour_tech_req = props.get("TechnikR")
        tour_type = props.get("Typ_TR")

        if tour_type == ttype:

            ls = kml.newlinestring(name="%s: %s" % (tour_nr, tour_name))
            ls.coords = list(geom)
            ls.style = styles.get(tour_type)

    kml.save("./Daten/Veloland-%se-Routen.kml" % ttype)

In [ ]:
print(kml.kml())